<a href="https://colab.research.google.com/github/Try-H4ck-M3/Try-H4ck-M3/blob/main/CodeReasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers --quiet

import tensorflow as tf
from datasets import load_dataset

In [2]:
from datasets import load_dataset

# You can pick either "split_0" or "split_1"
dataset = load_dataset("nvidia/OpenCodeReasoning", "split_0", split="split_0")

# Print one example to inspect
print(dataset[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

{'id': 'c0d4209f929db2b5bf3526a47a2520b0', 'input': 'Problem description.\nVipul is a hardworking super-hero who maintains the bracket ratio of all the strings in the world. Recently he indulged himself in saving the string population so much that he lost his ability for checking brackets (luckily, not permanently ).Being his super-hero friend\xa0help him in his time of hardship.\nInput\n\nThe first line of the input contains an integer T denoting the number of test cases. The description of T test cases follows.\nThe first line of each test case contains a single string S denoting the string to be checked.\n\n\nOutput\n\nFor each test case, output a single line printing "YES" or "NO" (without " " and in uppercase only) , denoting if the brackets in the given string is balanced or not .\n\n\nConstraints\n\n1 ≤ T ≤ 10\n1 ≤ length of S ≤ 60\n\n\nExample\nInput:\n3\n((()))\n(())()\n()(()\n\nOutput:\nYES\nYES\nNO\n\n\xa0\n\nExplanation\nExample is self-explanatory.', 'output': '<think>\nOk

In [ ]:
# Combine question and solution into a single text block for language modeling
texts = [f"Question: {item['input']}\nSolution:\n{item['output']}" for item in dataset]

# Create a tokenizer (character-level for now, or you can swap with a subword tokenizer)
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Create input/target sequences
seq_length = 256  # Adjust based on memory
inputs = []
targets = []

for seq in sequences:
    for i in range(len(seq) - seq_length):
        inputs.append(seq[i:i+seq_length])
        targets.append(seq[i+1:i+seq_length+1])

# Convert to TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(10000).batch(64)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 256
rnn_units = 1024

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    tf.keras.layers.GRU(rnn_units, return_sequences=True),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy')
model.fit(dataset, epochs=10)

In [ ]:
def generate_code(prompt, tokenizer, model, seq_length, num_generate=200):
    input_ids = tokenizer.texts_to_sequences([prompt])[0]
    input_ids = input_ids[-seq_length:]
    input_ids = tf.keras.preprocessing.sequence.pad_sequences([input_ids], maxlen=seq_length)

    generated = []

    for _ in range(num_generate):
        preds = model.predict(input_ids, verbose=0)
        pred_id = tf.argmax(preds[0][-1]).numpy()
        generated.append(pred_id)
        input_ids = tf.concat([input_ids[:, 1:], [[pred_id]]], axis=1)

    return tokenizer.sequences_to_texts([generated])[0]

# Example
prompt = "Question: Write a Python function to check if a number is prime.\nSolution:\n"
print(generate_code(prompt, tokenizer, model, seq_length))